In [106]:
from selenium import webdriver
# from selenium.webdriver.firefox.options import Options
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import langchain_ollama

from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
import time
from datetime import datetime


MY_NAME = "Vikas Reddy"

#### Creating a selenium based chrome driver for actions

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager  # optional, but handy

options = Options()
# point at the running Chrome on 9222
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

# if your Chrome binary isn’t on PATH, also do:
# "C:\Program Files\Google\Chrome\Application\chrome.exe" --remote-debugging-port=9222 --user-data-dir="C:\ChromeDebug" --no-first-run --no-default-browser-check

print("Using Chrome binary:", options.binary_location)
# pick a driver that matches your Chrome version:
service = Service(ChromeDriverManager().install())

print("Using ChromeDriver binary:", service.path)
# this will attach instead of launching a new window
driver = webdriver.Chrome(service=service, options=options)

print("Attached to Chrome, current URL:", driver.current_url)
driver.get("https://www.linkedin.com/search/results/people")



Using Chrome binary: 
Using ChromeDriver binary: C:\Users\VIKAS\.wdm\drivers\chromedriver\win64\136.0.7103.92\chromedriver-win32/chromedriver.exe
Attached to Chrome, current URL: https://www.linkedin.com/messaging/thread/2-YWY4ZjY3MWEtOWExNy00NTYwLWFkMTctOWY2YTBiMmE4ZTU5XzAxMg==/


In [19]:
## click on the message bar

def click_on_message():
    message_bar = driver.find_element(By.XPATH, "//span[normalize-space(.)='Messaging']")
    message_bar.click()

click_on_message()  

In [20]:
# scrolls upwards for the top of the page for the user

def wheel_scroll_up(amount=-600, repeats=5, pause=0.10):
    """
    Spin the mouse wheel upward inside the message list.

    amount   – vertical pixels per wheel tick  (negative = up, positive = down)
    repeats  – how many wheel ticks to send
    pause    – delay between ticks so LinkedIn can lazy-load older messages
    """
    # 1. locate the scrollable <div>
    scroll_el = driver.find_element(
        By.CSS_SELECTOR, ".msg-s-message-list.full-width.scrollable"
    )

    # 2. define where the wheel should originate (center of the element)
    origin = ScrollOrigin.from_element(scroll_el)

    # 3. build the wheel actions
    actions = ActionChains(driver)
    for _ in range(repeats):
        actions.scroll_from_origin(origin, 0, amount)
        actions.pause(pause)        # give time for lazy-loading

    actions.perform()

# example: yank the scrollbar up ~3 000 px in 5 smooth “ticks”



In [21]:
## this function is check if messages time into a particular format

def is_valid_datetime_format(date_string):
    date_string = date_string.strip()  # Remove leading/trailing whitespace
    try:
        datetime.strptime(date_string, "%b %d, %Y %I:%M %p")
        return True
    except ValueError:
        return False

# sanity
# is_valid_datetime_format(" Apr 5, 2024 11:31 PM")  # True

In [22]:
# this function is to get the time of the message

def time_formatter(raw):# 1) Strip leading/trailing whitespace
    raw_clean = raw.strip()

    # 2) Parse into a datetime object (24-hour output by default)
    dt = datetime.strptime(raw_clean, "%b %d, %Y %I:%M %p")
    return dt

In [ ]:
def get_time(event, prev):
    # 1) Find the <time> node inside this <li>
    time_elems = event.find_elements(By.TAG_NAME, "time")

    time_value = ""
    time_count = 0

    for i in time_elems:
        try:
            if datetime.strptime(i.text, "%b %d"):
                time_value = i.text + f", {datetime.now().year}"
        except :
            time_value += f" {i.text}"
        time_count += 1
    

    if time_count <= 1:
        print("here before", time_value, prev)
        time_value = f" {prev.split()[0]} {prev.split()[1]} {prev.split()[2]}" + time_value
        print("here after", time_value, prev)
    
    prev = time_value
    
    # Convert "THURSDAY 9:13 PM" or "TODAY 9:13 PM" to "%b %d, %Y %I:%M %p" format
    if time_value.split()[0].upper() in ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY", "TODAY"]:
        day_of_week_or_today = time_value.split()[0].upper()
        time_part = time_value.split()[1] + " " + time_value.split()[2]
        today = datetime.now()
        
        if day_of_week_or_today == "TODAY":
            recent_date = today
        else:
            days_ago = (today.weekday() - ["MONDAY", "TUESDAY", "WEDNESDAY", "THURSDAY", "FRIDAY", "SATURDAY", "SUNDAY"].index(day_of_week_or_today)) % 7
            recent_date = today - timedelta(days=days_ago)
        
        time_value = recent_date.strftime("%b %d, %Y") + " " + time_part
    
    return time_value, prev


def get_title(event):
    
    a_tag = event.find_element(By.XPATH, ".//a[img]")
    img_tag = a_tag.find_element(By.TAG_NAME, "img")
    title = img_tag.get_attribute("title")
    
    return title


def get_name_of_the_connection():

    a_tag_with_open = driver.find_element(By.XPATH, "//a[contains(@title, 'Open')]")
    name = a_tag_with_open.get_attribute("title")
    name = name.replace("’s profile", "").replace("Open ", "")  # Handle curly apostrophe as well
    print("name of the user:", name)
    return name


In [221]:

from datetime import datetime

def format_datetime(dt: datetime) -> str:
    """
    Convert a datetime object to 'YYYY-MM-DD HH:MM:SS' string.
    """
    return dt.strftime("%Y-%m-%d %H:%M:%S")


# Usage example:
dt = datetime(2025, 5, 2, 9, 14)
print(format_datetime(dt))  # outputs: 2025-05-02 09:14:00


2025-05-02 09:14:00


In [235]:
import os, json
# Initialize a list to store messages in the desired format
# Check if the file exists
file_name = "lnkdn.json"
if os.path.exists(file_name):
    # Read the existing data
    with open(file_name, "r") as json_file:
        messages_data = json.load(json_file)
else:
    messages_data = []


unique_dates_and_conversations = set((message['message_date'], message['conversation_id'], message["connection_name"]) for message in messages_data)


In [244]:
from datetime import timedelta


def retrieve_profile_messages(message_id):

    # go to the top of the conversation
    wheel_scroll_up(amount=-600, repeats=5)

    event_elements = driver.find_elements(By.XPATH, "//li[contains(@class, 'msg-s-message-list__event clearfix')]")
    
    name = get_name_of_the_connection()

    
    prev_message = None
    p_tag_text = ""
    prev = ""
    prev_time = ""

    for indx, event in enumerate(event_elements):
        
        # 1. get the title of the user. 
        div_with_title = driver.find_element(By.XPATH, ".//div[@title]")
        print("title :: ", div_with_title.get_attribute("title"))
        
        # 2. get time_value and prev.
        time_value, prev = get_time(event, prev)

        # 3. get the text values from the table. 
        p_tag = event.find_element(By.TAG_NAME, "p") if event.find_elements(By.TAG_NAME, "p") else None
        p_tag_text = p_tag.text if p_tag else ""

        # 4. get the name of the person sent the message
        try:
            span_elem = event.find_element(By.XPATH, ".//span[contains(@class, 'msg-s-event-listitem--group-a11y-heading')]")
            driver.execute_script("arguments[0].scrollIntoView(true);", span_elem)  # Scroll into view if necessary
            message_sender = driver.execute_script("return arguments[0].innerText;", span_elem)  # Use JavaScript to retrieve innerText
            print("message_sender:", message_sender)
        except Exception as e:
            print("message_sender", message_sender)

        print(time_value, prev, is_valid_datetime_format(time_value))
        if is_valid_datetime_format(time_value):
            msg_time = time_value
            prev_time = time_value
        else:
            msg_time = prev_time
        message_sender = message_sender.split(" sent ")[0]

        temp_time = time_formatter(msg_time)
        message = {
        "message_id": f"messg_{message_id}",
        "conversation_id": f"conv_{indx}",
        "platform": "LinkedIn",
        "connection_name": name,
        "message_sender": message_sender,
        "message_date": temp_time,
        "message_text": p_tag_text,
        "connection_title": div_with_title.get_attribute("title")
        }
            
        # Convert datetime object to the desired string format
        
        # print(message["message_date"], message["conversation_id"], "new time - -------------------------------------------------")  # Output: 2025-05-02 09:14:00
        if (format_datetime(message["message_date"]), message["conversation_id"], message["connection_name"]) not in unique_dates_and_conversations:

            print("in :::::::::::::::::::", (type(message["message_date"]), message["conversation_id"]))
            messages_data.append(message)
            unique_dates_and_conversations.add((format_datetime(message["message_date"]), message["conversation_id"], message["connection_name"]))
        else:
            print("message already present:: skipping")
        
    print(p_tag_text)
    print("----------------------------------------------------------------------------------")
        
        

In [238]:
retrieve_profile_messages(1)
# messages_data

name of the user: Xiaona (Eva) Ou
title ::  Sales Analyst at Medline Industries, Inc.
message_sender: Vikas Reddy Venkannagari sent the following messages at 7:34 PM
MAY 2, 2025 7:34 PM MAY 2, 2025 7:34 PM True
2025-05-02 19:34:00 conv_0 new time - -------------------------------------------------
message already present:: skipping
Hey Xiaona, I’m Vikas, a current grad student at UMBC, applying for Data Scientist role at Medline Industries. I’d love your take—would you be open to a referral? I bring over 1.5 years of relevant experience and am happy to share my resume. Thanks!
----------------------------------------------------------------------------------


In [260]:
import os, json
# Initialize a list to store messages in the desired format
# Check if the file exists
file_name = "lnkdn.json"
if os.path.exists(file_name):
    # Read the existing data
    with open(file_name, "r") as json_file:
        messages_data = json.load(json_file)
else:
    messages_data = []


unique_dates_and_conversations = set((message['message_date'], message['conversation_id'], message["connection_name"]) for message in messages_data)


In [261]:
# clicks on the message bar
# click_on_message()
import time
# goes through each profile in the profiles bar
def retrieve_messages():
    index = 0

    conversation_list = driver.find_element(By.XPATH, "//ul[@aria-label='Conversation List']")

    for item in conversation_list.find_elements(By.TAG_NAME, "li"):
        if index < last_message_id:
            index += 1
            continue
        try:
            item.click()
            time.sleep(2)
            x = retrieve_profile_messages(index)
            index += 1
            time.sleep(random.randint(4, 6))
        except Exception as e:
            print("Error:", e)
            continue
        time.sleep(random.randint(4, 6))
        print(messages_data[-1])

        

In [262]:
retrieve_messages()

name of the user: Mingxiu Du
title ::  Senior Associate, Early Careers Recruiting
message_sender: Vikas Reddy Venkannagari sent the following messages at 1:53 AM
 NOV 4, 2024 1:53 AM  NOV 4, 2024 1:53 AM True
message already present:: skipping
Hello Mingxiu Du,
 
I hope you’re well! I’m reaching out to express my interest in early career opportunities as a Software Engineer. I previously worked with EPAM Systems as a Software Engineer, and I’m currently pursuing my Master’s in Data Science, where I also work as a Research Assistant applying machine learning to brain-computer interfaces (BCIs).
 
I would love the chance to interview and discuss how my skills and background could contribute to your team. I’ve attached my resume for your review.
 
Thank you very much for your time and consideration!
----------------------------------------------------------------------------------
{'message_id': 'messg_176', 'conversation_id': 'conv_13', 'platform': 'LinkedIn', 'connection_name': 'Abhinav

In [257]:
# messages_data
unique_dates_and_conversations

{('2024-04-04 00:16:00', 'conv_0', 'Maddipati Venkata Akarsh'),
 ('2024-04-04 00:17:00', 'conv_1', 'Maddipati Venkata Akarsh'),
 ('2024-04-05 23:31:00', 'conv_0', 'Sameer Kibey'),
 ('2024-04-06 13:54:00', 'conv_1', 'Sameer Kibey'),
 ('2024-04-06 15:25:00', 'conv_2', 'Sameer Kibey'),
 ('2024-04-09 23:54:00', 'conv_3', 'Sameer Kibey'),
 ('2024-06-02 17:12:00', 'conv_4', 'Sameer Kibey'),
 ('2024-06-02 17:12:00', 'conv_5', 'Sameer Kibey'),
 ('2024-06-26 00:30:00', 'conv_0', 'Poojitha Emani'),
 ('2024-07-13 15:56:00', 'conv_6', 'Sameer Kibey'),
 ('2024-07-13 21:56:00', 'conv_7', 'Sameer Kibey'),
 ('2024-07-13 21:57:00', 'conv_8', 'Sameer Kibey'),
 ('2024-07-16 20:37:00', 'conv_9', 'Sameer Kibey'),
 ('2024-07-16 20:38:00', 'conv_10', 'Sameer Kibey'),
 ('2024-07-16 20:38:00', 'conv_11', 'Sameer Kibey'),
 ('2024-09-28 08:51:00', 'conv_0', 'Abhinav Varma V'),
 ('2024-09-28 08:53:00', 'conv_1', 'Abhinav Varma V'),
 ('2024-09-28 13:00:00', 'conv_2', 'Abhinav Varma V'),
 ('2024-09-28 13:42:00', 'c

In [263]:
import json
import os

file_name = "lnkdn.json"

# Check if the file exists
if os.path.exists(file_name):
    # Read the existing data
    with open(file_name, "r") as json_file:
        existing_data = json.load(json_file)
else:
    existing_data = []

# Append the new messages
existing_data.extend(messages_data)

# Write the updated data back to the file
with open(file_name, "w") as json_file:
    json.dump(existing_data, json_file, indent=4, default=str)  # Use default=str to handle datetime serialization


In [259]:
import json

# Read the lnkdn.json file
file_name = "lnkdn.json"
with open(file_name, "r") as json_file:
    data = json.load(json_file)

# Count the number of unique messg_id
last_message_id = {message['message_id'] for message in data}
last_message_id = len(last_message_id)
print(f"Number of unique messg_id: {last_message_id}")

Number of unique messg_id: 175
